In [ ]:
!pip install transformers[sentencepiece]
from transformers import pipeline
summarizer = pipeline("summarization", min_length=5, max_length=20, model="sshleifer/distilbart-cnn-6-6")
translator = pipeline("translation_grk_to_en", model='Helsinki-NLP/opus-mt-grk-en')
retranslator = pipeline("translation_en_to_el", model='Helsinki-NLP/opus-mt-en-el')

In [ ]:
!pip install rouge-score

In [ ]:
summarizer = pipeline("summarization", min_length=5, max_length=50, model="sshleifer/distilbart-cnn-6-6")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder='/content/drive/My Drive/multilingMss2015Training/'
!ls '/content/drive/My Drive/multilingMss2015Training/'

import os


Mounted at /content/drive
announcement.txt  body	summary  target-length


In [ ]:
def split_strings_within_limit(strings, max_length=1000):
    result = []
    current_string = ""

    for string in strings:
        if len(string) <= max_length:
            # If the current string plus the new string is within the limit, concatenate
            if len(current_string) + len(string) <= max_length:
                current_string += string
            else:
                # If adding the new string would exceed the limit, start a new string
                result.append(current_string)
                current_string = string
        else:
            # If the new string itself is longer than the limit, split it into chunks
            for i in range(0, len(string), max_length):
                result.append(string[i:i + max_length])

    if current_string:
        result.append(current_string)

    return result

In [ ]:
import csv

def read_csv_to_dict(file_path):
    data_dict = {}

    with open(file_path, mode='r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)

        for row in csv_reader:
            if len(row) == 2:
                key, value = row
                data_dict[key] = value

    return data_dict

# Example usage
csv_file_path = folder+'target-length/el.txt'
target_sizes = read_csv_to_dict(csv_file_path)
print(target_sizes)

{'0218db401253e76a84a568453a7cda5f_body.txt': '1296', '0d86383293996518eee416037c5ff576_body.txt': '1127', '0e697d0ffbed773c79e449f71c4cf9bb_body.txt': '707', '0ed50e7a7058171bf0305aa9ee33dd8d_body.txt': '1174', '15093808572f2be8c394a3acddf7a5e6_body.txt': '1319', '242424914004d4870b483c72b9c949ba_body.txt': '823', '27abeb23b904ec662ac2af636235cf35_body.txt': '1322', '31036d7e299c27bda3b83300ef8d44d6_body.txt': '938', '39f64462c48b3ab3fb053b671104ec2b_body.txt': '1061', '3aa6e4a33f78a86fdd17620156238fca_body.txt': '1218', '3b3d000f9dca943f2bb2c33e31fa33f0_body.txt': '1444', '3ebf88a88c54039d9bf33596b362fa95_body.txt': '891', '430551a29ade3ec53eac2799b6186928_body.txt': '1299', '5a2b3ed3aa2fa2d3cee2fb533fb24111_body.txt': '1114', '5a6c1f722178dc60dbfc9bee0bf3a0b0_body.txt': '1232', '6613e82f5ace4791e571c2f9b8da1e23_body.txt': '953', '6bdaa262298a41f78ede6889521b242c_body.txt': '840', '6e9bfd57cf631d844410e61082b8997d_body.txt': '1188', '7e2e7ed635d9efab20cd0d1decd846ab_body.txt': '1674'

In [ ]:
def doc_length(text_tokens):
  length=0
  for t in text_tokens:
    length+=len(t)
  return length


In [ ]:
GreekSummaries={}
body_folder= folder+'body/text/el/'
folder_contents = os.listdir(body_folder)
for file in folder_contents:
  target_length=int(target_sizes[file])
  print(target_length)
  file_path = body_folder+file
  print(file_path)
  try:
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        # Remove newline characters from the end of each line
        lines = [line.strip() for line in lines]
        print(len(lines))
  except FileNotFoundError:
      print(f"File not found: {file_path}")
  except Exception as e:
      print(f"An error occurred: {e}")

  preprocessed_lines=[]
  for line in lines:
    line=line.replace('[Επεξεργασία]','')+' .'
    preprocessed_lines.append(line)

  # grouped_lines=split_strings_within_limit(preprocessed_lines,100)
  # translated_tokens=translator(grouped_lines[0:2])

  grouped_lines=split_strings_within_limit(preprocessed_lines,1000)
  translated_tokens=translator(grouped_lines)

  translations = []
  for value in translated_tokens:
    print(value)
    translations.append(value['translation_text'])
  summarized_tokens = summarizer(translations)
  print(summarized_tokens)
  english_summaries=[]
  for value in summarized_tokens:
    print(value)
    english_summaries.append(value['summary_text'])

  english_summaries2=[]
  if doc_length(english_summaries)>target_length:
    grouped_summary_lines=split_strings_within_limit(english_summaries,target_length)
    summarized_tokens2 = summarizer(grouped_summary_lines)
    print(summarized_tokens2)
    for value in summarized_tokens2:
      print(value)
      english_summaries2.append(value['summary_text'])


  if len(english_summaries2)>0:
    greek_sum=retranslator(english_summaries2)
  else:
    greek_sum=retranslator(english_summaries)

  Greek_Summary=''
  for value in greek_sum:
    Greek_Summary+=value['translation_text']+' '

  print(Greek_Summary)
  GreekSummaries[file]=Greek_Summary
  # break


In [ ]:
GreekSummaries

In [ ]:
print(GreekSummaries)